In [ ]:
NAME = "" # put your full name here
COLLABORATORS = [] # list names of anyone you worked with on this homework.

# [ER-131] Homework 3: EDA Fire Incident Data
<br>

### Table of Contents
[Introduction](#intro)<br>
1 - [The IOU data](#data)<br>
2 - [Merging IOU and Weather Station Data](#merge)<br>
3 - [EDA](#eda)<br>
4 - [Exploring data through tables and visuals](#tables_plots)<br>
5 - [Summarizing data](#summarize)<br>

### Introduction <a id='intro'></a>

In this homework, you will investigate fire incident data from the three California Investor Owned Utilities (IOUs). The main goal for this assignment is to establish different ways to explore your data and its limitations, as well as ways to summarize and re-organize data.

We will accomplish this by utilizing exploratory data analysis (EDA) to analyze utility-reported data alongside weather data.

### Learning Objectives 

* Demonstrate your ability to work with different file types
* Merge dataframes and perform operations to add new columns
* Explain the structure, granularity, scope, temporality and faithfulness of different data
* Perform basic data cleaning operations

### Dependencies

**Question 0 (1 pt):** Import the NumPy, Pandas, matplotlib, and Geopandas packages using their common pseudonyms. 

In [ ]:
# YOUR CODE HERE

In [ ]:
# Run this cell to import a few more packages.
!pip install pyogrio
import csv
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

from IPython.display import display, Latex, Markdown
pd.set_option('display.max_columns', 36)

from shapely.geometry import Point

----
## Section 1: The IOU Data<a id='data'></a>

In this notebook, you'll be working with data from the [California Public Utilities Commission](https://www.cpuc.ca.gov/industries-and-topics/wildfires). The three California Investor-Owned Utilities – IOUs – (PG&E, SCE, and SDGE) are required to report fire incidents to the CPUC, along with certain characteristics of the fire and the electrical system in the area.

<br>**Question 1.1a (1 pt):** Look through the `data` folder and then read the Shapefile into this notebook using the geopandas library. The csvs were retrieved from the CPUC website, and the first (PG&E) was converted into a shapefile. 

Take a look at the arguments that are passed to the `read_file` function. First, we specify the file location. We also set  `index_col` to `False`.  This forces numbered indices. As an alternative, we could have passed a number to `index_col`; if we pass $n$ in, then Pandas uses the $n+1^{\text{st}}$ column of the csv as the index. 

In [ ]:
# YOUR CODE HERE
# TEMPLATE: pge = gpd.read_file(..., index_col=False, engine = 'pyogrio')


In [ ]:
pge.columns

Geopandas truncates column names that exceed 10 characters. Run the chunk below to replace the column names with their original names. 

In [ ]:
# Run this cell
raw = pd.read_csv('data/PGE_FireIncidentData_2014-2019.csv')
names = list(raw.columns) + ['geometry'] # We added a geometry column to convert the csv data to a GeoDataFrame, so we also need to add the name of this column
# The GeoDataFrame no longer contains the lat/lon fields that the csv contains
names.remove('Latitude') 
names.remove('Longitude') 
pge.columns = names

In [ ]:
pge.columns

**Question 1.1b (1 pt):** What type of geometry are the objects in the `pge` GeoDataFrame?

In [ ]:
# scratch work here

*Your answer here*

**Question 1.2a (1 pt)** In our `data` folder, we have a Shapefile only for PG&E data, but we do have csvs for Southern California Edison (SCE) and San Diego Gas & Electric (SDGE) that contain Latitude and Longitude information. Read in those SCE and SDGE csvs in:

In [ ]:
# YOUR CODE HERE
sce_df = ...
sdge_df = ...

In [ ]:
sce_df.head()

In [ ]:
sdge_df.head()

**Question 1.2b (1 pt)** Check if the columns of these two pandas DataFrames are the same. Are they the same as the columns of the `pge` GeoDataFrame? (You can do this check by visually examining the column names, or you may find the [numpy.setdiff1d](https://numpy.org/doc/stable/reference/generated/numpy.setdiff1d.html) method helpful) 

In [ ]:
# scratch work here

*YOUR ANSWER HERE*

Run the two chunks below to construct our new GeoDataFrames, `sce` and `sdge` from corresponding DataFrames (and note that in the process, we're choosing to drop the 'Longitude' and 'Latitude' fields from `sce_df` and `sdge_df`). 

In [ ]:
# SCE
sce_geometry = [Point(xy) for xy in zip(sce_df.Longitude, sce_df.Latitude)]
sce_df = sce_df.drop(['Longitude', 'Latitude'], axis=1)
sce = gpd.GeoDataFrame(sce_df, crs="EPSG:4326", geometry=sce_geometry)

In [ ]:
# SDGE
sdge_geometry = [Point(xy) for xy in zip(sdge_df.Longitude, sdge_df.Latitude)]
sdge_df = sdge_df.drop(['Longitude', 'Latitude'], axis=1)
sdge = gpd.GeoDataFrame(sdge_df, crs="EPSG:4326", geometry=sdge_geometry)

In [ ]:
print(type(sce))
print(type(sdge))

**Question 1.3a (1pt)** When we examine the values for the `Equipment Type` or `Type` columns in `pge`, `sce`, and `sdge` where they appear, it seems that these column names indicate the same type of information. In whichever of the `pge`, `sce`, or `sdge` GeoDataFrames in which the `Type` column appears, let's rename it to the more specific column name `Equipment Type` for consistency across GeoDataFrames. *Hint: Consider using the rename method:* `DataFrame.rename` 

In [ ]:
# YOUR CODE HERE


In [ ]:
assert sdge.columns.all() == sce.columns.all()
assert sdge.columns.all() == pge.columns.all()
assert pge.columns.all() == sce.columns.all()
assert len(sce.columns) == 23

**Question 1.3b (1 pt):** What type of geometry are the objects in the `pge`, `sce`, and `sdge` GeoDataFrames?

In [ ]:
# scratch work here

*YOUR ANSWER HERE*

Let's plot the three IOU datasets on the same map to get a sense of where the data lie.

In [ ]:
# Run this cell
fig, ax = plt.subplots(figsize = (12,8))
pge.plot(ax = ax, label = 'PG&E', color = 'gold', marker = '*', markersize = 50)
sce.loc[1:10].plot(ax = ax, label = 'SCE', color = 'maroon', marker = 'o', markersize = 50)
sdge.plot(ax = ax, label = 'SDG&E', color = 'teal', marker = '^', markersize = 50)
plt.legend(loc = 'lower center', bbox_to_anchor=(0.5, -4))
plt.title('Fire incidents in IOU territories')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show();

Uh oh! Looks like something is wrong with some of our data.

**Question 1.4a (1 pt):** Examine the graph carefully and describe a possible data error that is leading to the unexpected output of our map. 

*YOUR ANSWER HERE*

**Question 1.5a (1 pt):** The most extreme latitude and longitude points in California are approximately as follows: 
* North: 42.0095 
* East -114.1312 
* South: 32.5341
* West: -124.4096 
 

Fill in the ellipses below to identify any points in the three utility Geodataframes that fall outside of California. Note: the pipe | symbol is the equivalent of an "or" statement. In other words, the `.loc` function in the code block below is filtering the `sdge` GeoDataFrame if *any* of the four specified criteria are true.


In [ ]:
sdge.loc[(sdge.geometry.y > ...) | # Fill in the ellipses to check that no points fall above the northernmost boundary, or...
         (sdge.geometry.x > ...) | # ...to the right of the easternnmost boundary, or...
         (sdge.geometry.y < ...) |  # ... below the southernmost boundary, or...
         (sdge.geometry.x < ...)]   # ... to the left of the westernmost boundary.

In [ ]:
# Repeat this process for PG&E...
pge.loc[...]

In [ ]:
# ... and for SCE.
sce.loc[...]

**Question 1.5b (1 pt):** How many points fall outside California's boundaries for each of the three ignition datasets?

*YOUR ANSWER HERE*

SDGE:

PGE:

SCE:

**Question 1.5c (1 pt):** For the sake of expediency, `drop` the extraneous points identified in Question 1.4 from their respective GeoDataFrames (better alternatives could be to research each of these fires and manually correct the location, or replace the point coordinates with null values). Make sure to specify the `labels`, `axis`, and `inplace` parameters correctly. You should not rename the dataframe. 

*Hint: the elipses for `labels = ` should include the indices of the rows you want to drop*

In [ ]:
# YOUR CODE HERE
sdge.drop(labels = ..., axis = ..., inplace = ...)

In [ ]:
assert len(sdge) == 159
assert len(pge) == 1986
assert len(sce) == 577

Let's try our plot again.

In [ ]:
# Run this cell
fig, ax = plt.subplots(figsize = (12,8))
pge.plot(ax = ax, label = 'PG&E', color = 'gold', marker = '*', markersize = 50)
sce.plot(ax = ax, label = 'SCE', color = 'maroon', marker = 'o', markersize = 50)
sdge.plot(ax = ax, label = 'SDG&E', color = 'teal', marker = '^', markersize = 50)
plt.title('Fire incidents in IOU territories')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.show()

Much better!

## Section 2: Merging IOU and Weather Station Data<a id='merge'></a>

We'll also be working with weather data from the National Oceanic and Atmospheric Administration (NOAA). [Daily Summary Data](https://www.ncdc.noaa.gov/cdo-web/datasets#GHCND) were obtained for one land-based weather station per IOU service area from January 2014 to December 2019. 

**Question 2.1 (1 pt)**: Load the file `noaa_dailysummary.csv` into a Pandas dataframe named `weather`.

In [ ]:
# YOUR CODE HERE
weather = ...

In [ ]:
weather.head()

We're going to be merging fire incident data between each IOU and a land-based weather station in that IOU's service area. There are three weather stations in the dataframe `weather`, as shown in the output below. 'SAN DIEGO INTERNATIONAL AIRPORT, CA US' is in SDG&E's service area, 'SACRAMENTO AIRPORT ASOS, CA US' is in PG&E's service area, and 'LOS ANGELES INTERNATIONAL AIRPORT, CA US' is in SCE's service area. <br>

In [ ]:
weather["NAME"].unique() # look at weather station values

**Question 2.2 (1 pt)** Since we're going to use the `merge()` function, we want the fields (i.e., columns) that we merge on to have the same name. Both the IOU and weather station data have a field for date, but these columns have different names between the two datasets. Rename the "Fire Start Data" column in `sdge`, `pge`, and `sce` to match the "DATE" column in the `weather` dataframe using the function `.rename()`. Set the `inplace` parameter so that you do not have to reassign the dataset to a new name.

In [ ]:
# YOUR CODE HERE

Finally, run the code below to convert the data type of all the date columns to `datetime`.<br>

In [ ]:
#Run this cell
for df in [pge, sce, sdge, weather]: # change date data type to datetime
    df["DATE"] = pd.to_datetime(df["DATE"], errors = "coerce")

**Question 2.3 (1 pt)**: Create three new dataframes - `weather_sdge`, `weather_pge`, and `weather_sce` - that correspond to just the weather data in that IOU's service area. (Hint: the `str_contains` method might come in handy.)

In [ ]:
# Replace ellipses with your code
weather_sdge = ...
weather_pge = ...
weather_sce = ...

In [ ]:
assert len(weather_sdge) == len(weather_pge) == len(weather_sce) == 2191

**Question 2.4 (1 pt)**: Merge each utility's fire incident and weather data and save the merged dataframes as `sdge_fireweather`, `pge_fireweather`, and `sce_fireweather`. The data should be merged on the date fields. 

In [ ]:
# Replace ellipses with your code
pge_fireweather = ...
sce_fireweather = ...
sdge_fireweather = ...

**Question 2.5 (1 pt):** What type of merge did you use in Question 2.4 (inner, outer, left, right)? Why did you choose this type?

*YOUR ANSWER HERE*

In [ ]:
assert pge_fireweather.shape[1] == sce_fireweather.shape[1] == sdge_fireweather.shape[1] == 30

**Question 2.6 (1 pt):** Compare the number of records in the merged `_fireweather` dataframes to the original `pge`, `sce`, and `sdge` dataframes. Did you lose any records when you performed the merge? Why or why not?

In [ ]:
#scratch work here

*YOUR ANSWER HERE*

Before combining data from all three IOUs, we'll run the following `assert` statements to make sure that the column names are the same across the three dataframes.

*Note*: because the reporting is standardized for these IOUs, and because of some Excel cleaning that was done beforehand, the column names should match up. But if you're working with a dataset where column names need to be changed, you can use the [`rename` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) or a value assigment (e.g., `sdge_fireweather.columns = sce_fireweather.columns` would set the columns of `sdge_fireweather` to be the same as those in `sce_fireweather`, as long as you were certain that the columns represented the same values).

In [ ]:
assert all(pge_fireweather.columns == sce_fireweather.columns)
assert all(sce_fireweather.columns == sdge_fireweather.columns)

Now, we can use [`concat()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html#pandas.concat) to combine all three dataframes into one, called `alliou`. Run the cell below to combine the three IOU dataframes. We want our new dataframe `alliou` to renumber the indices (otherwise we'd have three rows with row index = 0, three rows with row index = 1, etc.). Check the [`concat()` documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html#pandas.concat) and make sure that we've set the appropriate argument to achieve this.

In [ ]:
alliou = pd.concat([pge_fireweather,sce_fireweather,sdge_fireweather], ignore_index = True)

In [ ]:
alliou.head()

In [ ]:
assert alliou.shape[0] > 2500
assert all(iou in alliou["Utility Name"].unique() for iou in ["PG&E", "SCE", "SDG&E"])

## Section 3: EDA<a id='eda'></a>

**Question 3.1 (5 pts):** 
Analyze the `alliou` table and see what data types are within the table. 
<br>What is the:
1. structure of the data?<br>
2. granularity of the data?<br>
3. scope of the data?<br>
4. temporality of the data?<br>
5. faithfulness of the data?<br>

Some questions to ask yourself:
* Structure - What was the format or file type of the imported data? Are there are any differences in data types between the individual IOU dataframes, the weather dataframe, and the combined dataframe?
* Granularity - What does each row of data represent? Do any of the fields represent aggregated data (data that is reduced or summarized in some way)? What's the resolution in time (eg. hourly, monthly) of the data?
* Scope - You can think of scope in different dimensions, but geographic scope and temporal scope is one place to start. Is the temporal scope the same for all three utilities?
* Temporality - What do the dates and times represent?
* Faithfulness - Where do the data come from? Is there any reason to question it? Where do you find null values? How have the manipulations we have conducted in this notebook revealed (and impacted) faithfulness?

Please make at least one observation for each category (structure, granularity, etc). The [NOAA's Daily Summary Documentation](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf) might be a helpful resource.

*YOUR ANSWER HERE*

**Question 3.2 (1 pt)**: To get a basic estimate of weather conditions on the day of the fire incident, we use daily summaries from one weather station in the service area of each IOU. This approach isn't particularly granular - the IOU datasets provide more detail both in terms of location and time than the weather data that we are using. Let's say you wanted to refine this approach to more effectively uncover the weather conditions as close as possible to the location and time of the fire incident. In a few sentences, qualitatively describe an alternative approach. <br>

You don't have to specify any code or functions, but you should reference which columns you would use (either in the IOU or weather datasets) and which datasets you would use - you can take a look at [available NOAA data here](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets).

*YOUR ANSWER HERE*

**Question 3.3 (2 pts):** What are the unique `Fire Size` categories in the `alliou` table? Describe any redundancies or ambiguities in how the fire sizes are bucketed. *Hint: Consider using the unique method: * `DataFrame['column_name'].unique()`

In [ ]:
# Scratch work here

*YOUR ANSWER HERE*

----
## Section 4: Exploring data through tables and visuals<a id='tables_plots'></a>

In this section, we'll do some data cleaning with the objective of exploring the fire incident data.


**Question 4.1 (2 pts)**: Use `pd.value_counts()` to get the number of reported fire incidents by utility. What are some factors that could explain the differences in number of reports between PG&E and SCE?

In [ ]:
# YOUR CODE HERE

*YOUR ANSWER HERE*

We might be interesting in understanding the seasonality of fires (i.e., when in the year fires occur) across the three IOU territories. Let's use the Pandas datetime functionality to add a column to the `alliou` called `day_of_year`. 

In [ ]:
# Run this cell
alliou['day_of_year'] = alliou['DATE'].dt.dayofyear
alliou.head()

**Question 4.2 (1 pt):** View the plots below with three histograms showing the number of fires that occur across the days of the year. Review the code for how to plot multiple plots within a single figure, to ensure that you could do this yourself. Using the plots below, describe how the seasonality of fires compares across the three IOU territories.

In [ ]:
fig, [ax0, ax1, ax2] = plt.subplots(nrows=1, ncols=3, figsize=(10,7))
ax0.hist(alliou.loc[alliou['Utility Name']=='PG&E']['day_of_year'],bins=12)
ax0.set_title('PG&E')
ax0.set_ylabel('Number of fires')

ax1.hist(alliou.loc[alliou['Utility Name']=='SCE']['day_of_year'],bins=12)
ax1.set_title('SCE')
ax1.set_xlabel('Day of year')

ax2.hist(alliou.loc[alliou['Utility Name']=='SDG&E']['day_of_year'],bins=12)
ax2.set_title('SDG&E')

plt.suptitle('Fire start day of year for reported CA IOU fires');

*YOUR OBSERVATIONS HERE*

**Question 4.3 (2 pts)**: Below, you will fill in the missing arguments to create a table showing the number of outage incidents associated with each type of material at the origin of the fire. Your table should have two columns ("no" and "yes") indicating whether there was an outage, and three rows ("Building", "Other", and "Vegetation") representing the material at origin types. It may help to look at the [documentation for `pd.pivot_table`](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

In [ ]:
# Examining the data
print(alliou["Was There an Outage"].unique())

# Cleaning the data
alliou["Was There an Outage"] = alliou["Was There an Outage"].str.lower()
alliou["Material at Origin"] = alliou["Material at Origin"].str.lower()

# Pivot Table
pd.pivot_table(
    data = alliou, 
    values= 'DATE', 
    index = ...,    # YOUR CODE HERE
    columns = ...,  # YOUR CODE HERE
    aggfunc='count'
)

<br>

----

## Section 5. Summarizing data<a id='summarize'></a>

One of the CPUC's goals when collecting this data is to identify operational and environmental trends related to fire incidents, with the objective of improving regulations and internal standards for utilities. In this section, you'll create two new dataframes: one that summarizes fire incident data by material at origin, and another that summarizes weather data by year. In the process, you'll gain more experience with using `.groupby()` along with summarizing data that is non-numerical or doesn't lend itself as well to `.groupby()`. 
<br>

**Question 5.1 (1 pt):** Define a new dataframe, `alliou_eqp`, that contains a single column containing every unique value for `Equipment Involved With` (this field refers to equipment involved with a fire ignition). Combine the conductor categories, and recategorize entries in the "None" category as "Unknown."

In [ ]:
# YOUR CODE HERE

# First, make a column in alliou for the cleaned equipment data.
# For the moment, we'll just copy the data from the "Equipment Involved With Ignition" column
alliou['Equipment_cleaned'] = alliou['Equipment Involved With']

# Fill in the ellipses to make sure there is only one category for conductors
alliou.loc[alliou['Equipment Involved With']==...,'Equipment_cleaned'] = ...

# Fill in the ellipses to recategorize "None" entries as "Unknown"
alliou.loc[alliou['Equipment Involved With'].isnull(),'Equipment_cleaned'] = ...

# Create a dataframe with one column
alliou_eqp = pd.DataFrame()
alliou_eqp["Equipment"] = ...

alliou_eqp

In [ ]:
assert len(alliou_eqp)==9

**Question 5.2 (1 pt):** The first set of values that we want to add to the dataframe is a count of the total number of fire incidents associated with each equipment category. Start by using `groupby().size()` to get a count of records for each equipment type and save it to variable `counts`.

In [ ]:
counts = ... # YOUR CODE HERE

In [ ]:
counts

**Question 5.3** (2 pt): Now we want to put the values from `counts` into a new column in dataframe `alliou_eqp`. Name this new column "Ignition Count." Do this below, making sure the right values from `counts` map to the correct equipment types. The resulting `alliou_eqp` dataframe should have two columns, one for equipment and one for the count of fire incidents.<br>

In [ ]:
# YOUR CODE HERE

In [ ]:
alliou_eqp

**Question 5.4 (1 pt)** Next, we want to find out what percentage of fire incidents involving each type of equipment are associated with outages, we want to add a column called "% Outage" to `alliou_eqp` that provides this value. There are lots of ways to approach finding the percentage of fire incidents associated with outages per material type, but some helpful functions might be `pivot_table()`, `groupby()` and `np.divide()`.

In [ ]:
#FILL IN THE ELIPSES
outages_df = pd.pivot_table(data = ..., 
                            values = ..., 
                            index = ..., 
                            columns = ...,
                            aggfunc = ...)



In [ ]:
# Run this cell to calculate outages associated with ignitions
outages_df['yes'][outages_df['yes'].isnull()] = 0
outage_counts = outages_df['yes']

alliou_eqp["% Outage"] = np.divide(outage_counts.values,alliou_eqp["Ignition Count"])*100

alliou_eqp

**Question 5.5 (2 pts)**: You're working for the CPUC, and as you're exploring the fire incident data a colleague notices that 88% of fire incidents involving switches are associated with outages. Your colleague concludes that a major focus of the commission should be to work with utilities to inspect and replaces switches in the utility territory. Do you agree with your colleague? Why or why not? Is there any additional data that you would want to collect before deciding where to focus maintenance review efforts? <br>

*YOUR ANSWER HERE*

**Question 5.6 (1 pt)** We also might like to explore seasonal trends. Create a new column in `alliou` called "Fire Start Month" that includes the year of the fire incident (the `.dt.month` method is helfpul here). Although we will not investigate seasonal trends in this assignment, we want to give you more familiarity with using datetime. 

In [ ]:
# YOUR CODE HERE

alliou["Fire Start Month"] = ...

In [ ]:
alliou.head()

----

## Submission

Congrats, you're done with homework 3!

Before you submit, click **Kernel** --> **Restart & Clear Output**. Then, click **Cell** --> **Run All**. Then, go to the toolbar and click **File** -> **Download as** -> **.pdf** and submit the file **as both an .pdf and .ipynb file through bCourses**.

----

## Bibliography

- CPUC Fire Incident Data Collection: https://www.cpuc.ca.gov/fireincidentsdata/
- NOAA Daily Summary Documentation: https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf